Importing required liabraries and dataset:

In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold

In [24]:
df=pd.read_csv("HR_comma_sep.csv")

In [25]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [26]:
df.isnull().any()

satisfaction_level       False
last_evaluation          False
number_project           False
average_montly_hours     False
time_spend_company       False
Work_accident            False
left                     False
promotion_last_5years    False
Department               False
salary                   False
dtype: bool

In [27]:
x=df.drop('left',axis=1)

In [28]:
x.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,0,sales,low
1,0.80,0.86,5,262,6,0,0,sales,medium
2,0.10,0.77,6,247,4,0,0,sales,low
3,0.92,0.85,5,259,5,0,0,sales,low
4,0.89,1.00,5,224,5,0,0,sales,low


In [29]:
y=df['left']

Using GridSearchCV to get best model.

Using Pipeline for One hot encoding with column selector as well as for KNN model

In [30]:
enc=OneHotEncoder(sparse_output=False).set_output(transform='pandas')

In [31]:
ct = make_column_transformer( (enc, make_column_selector(dtype_include=object) ),
       ("passthrough", make_column_selector(dtype_exclude=object)  ),
       verbose_feature_names_out=False).set_output(transform='pandas')

In [32]:
knn= KNeighborsClassifier()

In [33]:
pipe=Pipeline([('CT',ct),('KNN',knn)])

In [34]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [35]:
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('onehotencoder',
                                    OneHotEncoder(sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x0000016053607470>),
                                   ('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001605357B3E0>)],
                     verbose_feature_names_out=False)),
  ('KNN', KNeighborsClassifier())],
 'transform_input': None,
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('onehotencoder',
                                  OneHotEncoder(sparse_output=False),
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000016053607470>),
                                 ('passthrough', 'passthrough',
                                  <sklearn.compose._column_transforme

In [36]:
params={ 'KNN__n_neighbors': [2,3,4,5]}

In [37]:
gscv=GridSearchCV(pipe,param_grid=params,cv=kfold,verbose=3,scoring='neg_log_loss')

In [38]:
gscv.fit(x,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ...............KNN__n_neighbors=2;, score=-1.264 total time=   0.4s
[CV 2/5] END ...............KNN__n_neighbors=2;, score=-1.179 total time=   0.7s
[CV 3/5] END ...............KNN__n_neighbors=2;, score=-1.205 total time=   0.3s
[CV 4/5] END ...............KNN__n_neighbors=2;, score=-1.041 total time=   0.3s
[CV 5/5] END ...............KNN__n_neighbors=2;, score=-1.020 total time=   0.4s
[CV 1/5] END ...............KNN__n_neighbors=3;, score=-1.062 total time=   0.3s
[CV 2/5] END ...............KNN__n_neighbors=3;, score=-0.907 total time=   0.2s
[CV 3/5] END ...............KNN__n_neighbors=3;, score=-1.051 total time=   0.1s
[CV 4/5] END ...............KNN__n_neighbors=3;, score=-0.804 total time=   0.2s
[CV 5/5] END ...............KNN__n_neighbors=3;, score=-0.843 total time=   0.2s
[CV 1/5] END ...............KNN__n_neighbors=4;, score=-0.854 total time=   0.1s
[CV 2/5] END ...............KNN__n_neighbors=4;, 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('onehotencoder',
                                                                         OneHotEncoder(sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000016053607470>),
                                                                        ('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001605357B3E0>)],
                                                          verbose_feature_names_out=False)),
                                       ('KNN', KNeighborsClassifier())]),
             param_grid={'KNN__n_neighbors': [2, 3, 4, 5]},
             scoring='neg_log_loss', verbose=3)

In [39]:
print(gscv.best_params_)
print(gscv.best_score_)

{'KNN__n_neighbors': 5}
-0.6681323458751205


In [40]:
test=pd.read_csv("tst_hr.csv")

In [41]:
test.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'Department', 'salary'],
      dtype='object')

In [42]:
test['Department'].unique()

array(['sales', 'technical', 'support'], dtype=object)

In [43]:
bestmodel=gscv.best_estimator_
bestmodel.predict(test)

array([1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [44]:
import pickle
hr_model=open("model_hr3.pickle",'wb')
pickle.dump(bestmodel,file=hr_model)
hr_model.close()